In [1]:
import sqlite3
import csv
import os
import pandas as pd

def convert_sqlite_to_csv(db_path, output_dir):
    """
    Convert all tables in a SQLite database to CSV files and save schema information.
    
    Args:
        db_path: Path to the SQLite database file
        output_dir: Directory to save CSV files and schema information
    """
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Connect to the database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Get all table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [table[0] for table in cursor.fetchall()]
    
    schema_info = {}
    
    # Process each table
    for table in tables:
        print(f"Processing table: {table}")
        
        # Get table schema
        cursor.execute(f"PRAGMA table_info({table})")
        columns_info = cursor.fetchall()
        
        # Save schema information
        schema_info[table] = []
        column_names = []
        
        for col in columns_info:
            col_id, name, dtype, not_null, default_val, is_pk = col
            column_names.append(name)
            schema_info[table].append({
                "name": name,
                "type": dtype,
                "not_null": bool(not_null),
                "default_value": default_val,
                "is_primary_key": bool(is_pk)
            })
        
        # Export data to CSV with proper escaping
        df = pd.read_sql(f"SELECT * FROM {table}", conn)
        csv_path = os.path.join(output_dir, f"{table}.csv")
        df.to_csv(csv_path, 
                 index=False,
                 escapechar='\\',  # Add escape character
                 encoding='utf-8',  # Specify encoding
                 quoting=csv.QUOTE_MINIMAL,  # Quote only when necessary
                 doublequote=True)  # Use double quotes for escaping quotes
        print(f"Exported {table} to {csv_path}")
    
    # Save schema information to a separate file
    schema_path = os.path.join(output_dir, "schema_info.csv")
    with open(schema_path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["table_name", "column_name", "data_type", "not_null", "default_value", "is_primary_key"])
        
        for table, columns in schema_info.items():
            for col in columns:
                writer.writerow([
                    table,
                    col["name"],
                    col["type"],
                    col["not_null"],
                    col["default_value"],
                    col["is_primary_key"]
                ])
    
    print(f"Schema information saved to {schema_path}")
    conn.close()

if __name__ == "__main__":
    database_path = "/users/azibaeir/Research/Benchmarking/project/vulnerability_dataset/database/database.sqlite"
    csv_output_directory = "csv_output"
    
    convert_sqlite_to_csv(database_path, csv_output_directory)

Processing table: vulnerabilities
Exported vulnerabilities to csv_output/vulnerabilities.csv
Processing table: sqlite_sequence
Exported sqlite_sequence to csv_output/sqlite_sequence.csv
Processing table: vulnerabilities_gemma2_9b
Exported vulnerabilities_gemma2_9b to csv_output/vulnerabilities_gemma2_9b.csv
Processing table: vulnerabilities_qwen2_5_coder_7b
Exported vulnerabilities_qwen2_5_coder_7b to csv_output/vulnerabilities_qwen2_5_coder_7b.csv
Processing table: vulnerabilities_codellama_7b
Exported vulnerabilities_codellama_7b to csv_output/vulnerabilities_codellama_7b.csv
Processing table: vulnerabilities_deepseek_r1_32b
Exported vulnerabilities_deepseek_r1_32b to csv_output/vulnerabilities_deepseek_r1_32b.csv
Processing table: vulnerabilities_deepseek_v2_16b
Exported vulnerabilities_deepseek_v2_16b to csv_output/vulnerabilities_deepseek_v2_16b.csv
Processing table: vulnerabilities_deepseek_coder_16b
Exported vulnerabilities_deepseek_coder_16b to csv_output/vulnerabilities_deepse